In [1]:
import torch
from torch import nn
from torch.nn import functional as F

from torch.nn import Conv2d

In [21]:
enc_features2 = torch.randn(1, 32, 256, 256)

dec_features2 = torch.randn(1, 64, 256, 256)

In [22]:
class Channel_Attention(nn.Module):
    def __init__(self, enc_in_channels, dec_in_channels):
        super().__init__()
        
        self.avg_pool = nn.AdaptiveAvgPool3d((1, None, None))
        self.max_pool = nn.AdaptiveMaxPool3d((1, None, None))
        self.conv2d_enc  = nn.Conv2d(enc_in_channels, 1, kernel_size=1)
        self.conv2d_dec  = nn.Conv2d(dec_in_channels, 1, kernel_size=1)
       
       
    def forward(self, enc_features, dec_features):
        # Input shape: (N, C, H, W)
        # N, C, H, W = x.shape

        # Average pooling across channels
        avg_out = self.avg_pool(enc_features)
        avg_out1 = self.avg_pool(dec_features)


        # Max pooling across channels
        max_out = self.max_pool(enc_features)
        max_out1 = self.max_pool(dec_features)

        # Conv2D to reduce channels to 1
        en_conv_out = self.conv2d_enc(enc_features)
        dec_conv_out = self.conv2d_dec(dec_features)
        
        concat_enc_features = torch.cat([avg_out, max_out, en_conv_out], dim=1)
        # concat_enc_features.shape
        
        concat_dec_features = torch.cat([avg_out1, max_out1, dec_conv_out], dim=1)
        # concat_dec_features.shape
        
        con_7x7 = Conv2d(in_channels=3, out_channels=1, kernel_size=7, padding=3)
        output = con_7x7(concat_enc_features)
        output.shape
        
        con2d7x7 = Conv2d(in_channels=3, out_channels=1, kernel_size=7, padding=3)
        output1 = con2d7x7(concat_dec_features)
        output1.shape

        output2 = output + output1
        output2.shape
        
        sigmoid = nn.Sigmoid()
        output3 = sigmoid(output2)
        output3.shape

        return output3 # Return shape: (N, 1, H, W)
    
    ###concat_enc_features.unsqueeze, concat_dec_features
    
    







In [23]:
model = Channel_Attention(enc_in_channels=32, dec_in_channels=64)
enc_dec_features = model(enc_features2, dec_features2)
enc_dec_features.shape




torch.Size([1, 1, 256, 256])